In [ ]:
#Install or call required packages
install.packages('mlbench')
install.packages('MASS')
install.packages('pROC')
install.packages('mlbench')
library(dplyr)

In [ ]:
#Read data
Churn <- read.csv("~/Churn_Train.csv")
View(Churn)

In [ ]:
#Replace any negative values with NA 
Churn$account_length <- replace(Churn$account_length, which(Churn$account_length < 0), NA)
Churn$number_vmail_messages <- replace(Churn$number_vmail_messages, which(Churn$number_vmail_messages < 0), NA)

In [ ]:
#Remove any entries where there are 3 or less values in the row. These entries are too incomplete 
#to use.
Churn <- Churn[rowSums(is.na(Churn)) < 3, ]

In [ ]:
#Replace remaining NA values with column mean.
for (i in 1:ncol(Churn)) {
    na<-which(is.na(Churn[,i]),)
    if (length(na)!=0) {
    Churn[na,i]<-mean(Churn[,i],na.rm = T)}}

In [ ]:
#Create dataframe of numeric value columns to check data distribution with histograms
df <- data.frame(Churn$account_length, Churn$total_day_minutes, Churn$total_day_calls,
                Churn$total_day_charge, Churn$total_eve_minutes, Churn$total_eve_calls,
                Churn$total_eve_charge, Churn$total_night_minutes, Churn$total_night_calls,
                Churn$total_night_charge, Churn$total_intl_minutes, Churn$total_intl_calls,
                Churn$total_intl_charge, Churn$number_customer_service_calls)

In [ ]:
#Evaluate data distribution of columns
hist.data.frame(df)

In [ ]:
#Recode values of churn, international_plan, area_code and vm to binerize
Churn <- Churn%>%
mutate(churn=recode(churn, "yes" = "1", "no" = "0"))

Churn <- Churn%>%
mutate(international_plan=recode(international_plan, "yes" = "1", "no" = "0"))

Churn <- Churn%>%
mutate(voice_mail_plan=recode(voice_mail_plan, "yes" = "1", "no" = "0"))

Churn <- Churn%>%
mutate(area_code=recode(area_code, "area_code_510" = "0", "area_code_415" = "1",
                       "area_code_408" = "2"))

In [ ]:
#Change newly binarized columns to numeric values
Churn[, c(3:5,20)] <- sapply(Churn[, c(3:5,20)], as.numeric)

In [ ]:
#Load prediction data
load("~/Downloads/Customers_To_Predict.RData")
View(Customers_To_Predict)

In [ ]:
#Preprocess data by binerizing values to match previous data
Customers_To_Predict <- Customers_To_Predict%>%
mutate(international_plan=recode(international_plan, "yes" = "1", "no" = "0"))

Customers_To_Predict <- Customers_To_Predict%>%
mutate(voice_mail_plan=recode(voice_mail_plan, "yes" = "1", "no" = "0"))

Customers_To_Predict <- Customers_To_Predict%>%
mutate(area_code=recode(area_code, "area_code_510" = "0", "area_code_415" = "1",
                       "area_code_408" = "2"))

In [ ]:
#Change columns that were binarized to numeric
Customers_To_Predict[, c(3:5)] <- sapply(Customers_To_Predict[, c(3:5)], as.numeric)

In [ ]:
#Create descision tree model, review summary metrics.
Mod1 <- rpart(churn~.,data=Churn,cp=0)
summary(Mod1)

In [ ]:
# Table of results
table(predict(Mod1,type='class'),Churn$churn)

In [ ]:
# Predict the new data from the Customers To Predict file; write a file to R directory of predictions
pred <- predict(Mod1, Customers_To_Predict)
write.csv(pred, 'results.csv')